In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from string import punctuation

import locale
locale.setlocale(locale.LC_ALL, '')

'en_GB.UTF-8'

In [2]:
def series_setdiff(s1,s2):
    diff = np.setdiff1d(s1.dropna().unique(), s2.dropna().unique())
    print('Length:', len(diff))
    return diff

def series_intersect(s1,s2):
    intersection = np.intersect1d(s1.dropna().unique(), s2.dropna().unique())
    print('Length:', len(intersection))
    return intersection

def df_strip(df,keep_punc_only=True):
    
    df = df.copy()
    
    for col in df.columns:
        if df[ col ].dtype == 'O':
            if keep_punc_only:
                
                mask = ( df[ col ].str.fullmatch( '[' + punctuation + ']+' ) & df[ col ].notna() )
                only_punc = df.loc[ mask, col ]
                
                df[ col ] = df[ col ].str.strip( ' \t\r\n' + punctuation )
                
                df.update( only_punc )
                
            else:
                df[ col ] = df[ col ].str.strip( ' \t\r\n' + punctuation )
        else:
            continue
            
    return df

In [3]:
pd.set_option('display.max_rows', 100)
na_values = [
    '',
    '#N/A',
    '#N/A N/A',
    '#NA',
    '-1.#IND',
    '-1.#QNAN',
    '-NaN',
    '-nan',
    '1.#IND',
    '1.#QNAN',
    '<NA>',
    'N/A',
    'NULL',
    'NaN',
    'n/a',
    'nan',
    'null'
]

In [4]:
aliases_ = {
    
    'country': {
        'United States':'USA',
        'United States of America': 'USA',
        'US': 'USA',
        'U.S.': 'USA',
        'U.S.A.': 'USA',
        "People's Republic of China": "China",
        "Czech Republic": "Czechia",
        "Kingdom of the Netherlands": "The Netherlands",
        "Former Yugoslav Republic of Macedonia": "North Macedonia",
        "Macedonia": "North Macedonia",
        "F.Y.R.O.M.": "North Macedonia",
        "FYROM": "North Macedonia",
        "Socialist Federal Republic of Yugoslavia": "Yugoslavia",
        "Kingdom of Albania": "Albania",
        "Great Britain": "United Kingdom",
        "Sapin": "Spain",
        "Q13431386": "Australia"
    },
    
    'city': {
        'Washington, DC': 'Washington DC',
        'Washington, D.C.': 'Washington DC',
        'Ōsaka': 'Osaka'
    },
    
    'admin_area': {
        'Quebec City': 'Quebec',
        'Ōsaka': 'Osaka Prefecture',
        'Osaka': 'Osaka Prefecture',
    }
}

## Countries

In [5]:
# countries = pd.read_csv('../warehouse/countryInfo.txt',
#                         sep='\t',
#                         skiprows=49,
#                         encoding='utf8',
#                         low_memory=False,
#                         na_values=na_values,
#                         keep_default_na=False)
# countries.to_csv('../warehouse/countryInfo.csv',index=False)

countries = pd.read_csv('../warehouse/countryInfo.csv',
                           encoding='utf8',
                           low_memory=False,
                           na_values=na_values,
                           keep_default_na=False)

In [6]:
countries['Country'] = countries['Country'].replace(aliases_['country'])

In [7]:
country_names = countries['Country'].values.tolist()

In [238]:
africa = countries.loc[countries['Continent']=='AF', 'Country'].values.tolist()
africa

['Angola',
 'Burkina Faso',
 'Burundi',
 'Benin',
 'Botswana',
 'Democratic Republic of the Congo',
 'Central African Republic',
 'Republic of the Congo',
 'Ivory Coast',
 'Cameroon',
 'Cabo Verde',
 'Djibouti',
 'Algeria',
 'Egypt',
 'Western Sahara',
 'Eritrea',
 'Ethiopia',
 'Gabon',
 'Ghana',
 'Gambia',
 'Guinea',
 'Equatorial Guinea',
 'Guinea-Bissau',
 'Kenya',
 'Comoros',
 'Liberia',
 'Lesotho',
 'Libya',
 'Morocco',
 'Madagascar',
 'Mali',
 'Mauritania',
 'Mauritius',
 'Malawi',
 'Mozambique',
 'Namibia',
 'Niger',
 'Nigeria',
 'Reunion',
 'Rwanda',
 'Seychelles',
 'Sudan',
 'South Sudan',
 'Saint Helena',
 'Sierra Leone',
 'Senegal',
 'Somalia',
 'Sao Tome and Principe',
 'Eswatini',
 'Chad',
 'Togo',
 'Tunisia',
 'Tanzania',
 'Uganda',
 'Mayotte',
 'South Africa',
 'Zambia',
 'Zimbabwe']

In [8]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ISO                 252 non-null    object 
 1   ISO3                252 non-null    object 
 2   ISO-Numeric         252 non-null    int64  
 3   fips                249 non-null    object 
 4   Country             252 non-null    object 
 5   Capital             246 non-null    object 
 6   Area(in sq km)      252 non-null    float64
 7   Population          252 non-null    int64  
 8   Continent           252 non-null    object 
 9   tld                 251 non-null    object 
 10  CurrencyCode        251 non-null    object 
 11  CurrencyName        251 non-null    object 
 12  Phone               247 non-null    object 
 13  Postal Code Format  161 non-null    object 
 14  Postal Code Regex   161 non-null    object 
 15  Languages           249 non-null    object 
 16  geonamei

## Administrative areas

In [9]:
us_states = pd.read_html('https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States',
                         parse_dates=True)[0]

us_states.columns = [
    'state',
    'code',
    'capital',
    'largestCity',
    'ratified',
    'population',
    'areaTotal_mi2',
    'areaTotal_km2',
    'areaLand_mi2',
    'areaLand_km2',
    'areaWater_mi2',
    'areaWater_km2',
    'nRepresentatives'
]

In [10]:
us_state_names = us_states['state'].values.tolist()

In [11]:
us_counties = pd.read_html('https://en.wikipedia.org/wiki/List_of_United_States_counties_and_county_equivalents',
                          parse_dates=True)[3]

us_counties.columns = ['county','state','population','admin_name']

In [12]:
us_county_names = us_counties['county'].values.tolist()

In [13]:
uk_country_names = ['England','Northern Ireland','Scotland','Wales']

In [14]:
uk_counties = pd.read_html('https://en.wikipedia.org/wiki/List_of_counties_of_the_United_Kingdom',
                           parse_dates=True)

In [15]:
en_counties = uk_counties[0]

In [16]:
ni_counties = uk_counties[1]

In [17]:
sc_counties = uk_counties[2]

In [18]:
wa_counties = uk_counties[3]

In [19]:
en_counties['County','County'].str.strip('\s♠')

0                              Avon
1      Bath and North East Somerset
2                      Bedfordshire
3                           Bedford
4                         Berkshire
                   ...             
107                       Yorkshire
108          Yorkshire, East Riding
109         Yorkshire, North Riding
110          Yorkshire, West Riding
111                            York
Name: (County, County), Length: 112, dtype: object

In [20]:
uk_county_names = pd.concat([en_counties['County','County'],
                             ni_counties['County','County'],
                             sc_counties['County'],
                             wa_counties['County','County']],
                            ignore_index=True,
                            sort=True).str.strip('\s♠').values.tolist()

## Cities
- continent
- country
- countryCode
- adminRegion
- city:
    - Plain-text
    - Accented
- lat
- lon
- pop.

### cities from geonames
http://download.geonames.org/export/dump/

In [22]:
fp = '../warehouse/cities500.txt'
usecols = [0,1,2,3,4,5,8,9,10,11,12,13,14,17,18]
col_names = {
    0:'id',
    1:'city',
    2:'city_ascii',
    3:'alternateNames',
    4:'lat',
    5:'lng',
    6:'featureClass',
    7:'featureCode',
    8:'iso2',
    9:'cc2',
    10:'admin1',
    11:'admin2',
    12:'admin3',
    13:'admin4',
    14:'population',
    15:'elevation',
    16:'elevationModel',
    17:'timezone',
    18:'modDate'
}

gn = pd.read_csv(fp,
                 sep='\t',
                 header=None,
                 usecols=usecols,
                 encoding='utf8',
                 low_memory=False,
                 na_values=na_values,
                 keep_default_na=False)

gn = gn.rename(columns=col_names).sort_values('population',ascending=False).drop_duplicates(['city','iso2'])

gn = gn.loc[gn['population'] > 0]

gn = pd.merge(gn,countries[['ISO','Country']].rename(columns={'ISO':'iso2','Country':'country'}),on='iso2',how='left')

gn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147414 entries, 0 to 147413
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              147414 non-null  int64  
 1   city            147414 non-null  object 
 2   city_ascii      147397 non-null  object 
 3   alternateNames  118808 non-null  object 
 4   lat             147414 non-null  float64
 5   lng             147414 non-null  float64
 6   iso2            147414 non-null  object 
 7   cc2             5027 non-null    object 
 8   admin1          147384 non-null  object 
 9   admin2          128790 non-null  object 
 10  admin3          78581 non-null   object 
 11  admin4          30686 non-null   object 
 12  population      147414 non-null  int64  
 13  timezone        147414 non-null  object 
 14  modDate         147414 non-null  object 
 15  country         147414 non-null  object 
dtypes: float64(2), int64(2), object(12)
memory usage: 19.1+ 

In [23]:
gn.loc[gn['population']==770].sort_values('country')

,id,city,city_ascii,alternateNames,lat,lng,iso2,cc2,admin1,admin2,admin3,admin4,population,timezone,modDate,country
117458,9972525,Bangholme,Bangholme,NaN,-38.03842,145.18690,AU,NaN,07,22670,NaN,NaN,770,Australia/Melbourne,2019-07-18,Australia
117490,2142822,Wuuluman,Wuuluman,NaN,-32.55000,149.08333,AU,NaN,02,18230,NaN,NaN,770,Australia/Sydney,2019-07-18,Australia
117428,2778340,Göpfritz an der Wild,Goepfritz an der Wild,"Gopfritz,Gopfritz an der Wild,Göpfritz,Göpfrit...",48.72497,15.40236,AT,AT,03,325,32505,NaN,770,Europe/Vienna,2018-07-28,Austria
117484,2771841,Maschl,Maschl,NaN,47.36667,13.20000,AT,NaN,05,504,50418,NaN,770,Europe/Vienna,2018-07-28,Austria
117409,2762580,Unterwald,Unterwald,NaN,48.01667,14.46667,AT,NaN,04,415,41514,NaN,770,Europe/Vienna,2018-07-28,Austria
117424,620959,Tatarka,Tatarka,"Tatarka,Татарка",53.25330,28.81930,BY,NaN,06,NaN,NaN,NaN,770,Europe/Minsk,2012-03-23,Belarus
117474,2787830,Roly,Roly,Philippeville (Roly),50.13554,4.53727,BE,NaN,WAL,WNA,93,93056,770,Europe/Brussels,2020-04-05,Belgium
117442,1805076,Jiuxijiang,Jiuxijiang,"Chiu-hsi-chiang,Jiuxijiang",27.65556,110.61361,CN,CN,11,NaN,NaN,NaN,770,Asia/Shanghai,2012-01-18,China
117447,1790151,Xiaojiangkou,Xiaojiangkou,"Hsiao-chiang-k'ou,Hsiao-chiang-k’ou,Xiaojiangkou",27.88028,110.44667,CN,CN,11,NaN,NaN,NaN,770,Asia/Shanghai,2012-01-18,China
117433,8604562,Velika Kosnica,Velika Kosnica,NaN,45.76094,16.08081,HR,NaN,20,3188243,NaN,NaN,770,Europe/Zagreb,2016-12-27,Croatia


In [24]:
city_names = gn['city'].values.tolist()
city_ascii = gn['city_ascii'].values.tolist()
city_altnames = ','.join(gn['alternateNames'].str.replace('|'.join(country_names),'').dropna().values).split(',')
city_aliases = city_names + city_ascii + city_altnames
# gn_zdup = gn.drop_duplicates('city',keep=False)
# city_names_zdup = gn_zdup['city'].values

In [126]:
city_to_country = dict(zip(city_names,gn['country'].values))

In [26]:
gn['iso2'].value_counts(normalize=True)

FR    0.100445
US    0.097623
MX    0.088567
IT    0.076146
DE    0.070529
        ...   
AQ    0.000007
GS    0.000007
JE    0.000007
NF    0.000007
GI    0.000007
Name: iso2, Length: 246, dtype: float64

In [27]:
gn.nunique()

id                147414
city              141773
city_ascii        140781
alternateNames    117861
lat               133280
lng               136226
iso2                 246
cc2                  109
admin1               592
admin2             12842
admin3             34066
admin4             26561
population         33161
timezone             397
modDate             3299
country              246
dtype: int64

## Bands, genres, and location of formation

### wikidata

In [28]:
[f for f in os.listdir("./WikiData") if f.endswith(".csv")]

['recordingartist_locationofformation.csv',
 'musicalgroup_locatedinadministrativeterritorialentity.csv',
 'musicalgroup_genre.csv',
 'recordingartist_countryofcitizenship.csv',
 'musicalgroup_countryoforigin.csv',
 'musicalgroup_locationofformation.csv',
 'musicalgroup_country.csv']

In [279]:
wd = pd.DataFrame(columns=['artist'])

for csv in [f for f in os.listdir("./WikiData") if f.endswith(".csv")]:
    usecols = [1,3]
    if 'tiveterr' in csv:
        usecols = None
    df = pd.read_csv('./WikiData/'+csv,usecols=usecols,header=0)
    c0 = df.columns[0]
    df = df.rename(columns={c0: 'artist'})
    df['artist'] = df['artist'].str.strip()
    wd = pd.merge(wd,df,how="outer")
    
wd.columns = ['artist','loc_of_formation','admin_area','genre','citizenship','country','_country']
wd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102369 entries, 0 to 102368
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   artist            102369 non-null  object
 1   loc_of_formation  21152 non-null   object
 2   admin_area        741 non-null     object
 3   genre             59827 non-null   object
 4   citizenship       2974 non-null    object
 5   country           67923 non-null   object
 6   _country          8787 non-null    object
dtypes: object(7)
memory usage: 6.2+ MB


#### combine country into one column

In [280]:
mask = wd['country'].isna()
wd.loc[mask, 'country'] = wd.loc[mask, '_country']

mask = wd['country'].isna()
wd.loc[mask, 'country'] = wd.loc[mask, 'citizenship']

wd = wd.drop(['citizenship','_country'],axis=1)

In [281]:
wd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102369 entries, 0 to 102368
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   artist            102369 non-null  object
 1   loc_of_formation  21152 non-null   object
 2   admin_area        741 non-null     object
 3   genre             59827 non-null   object
 4   country           73326 non-null   object
dtypes: object(5)
memory usage: 4.7+ MB


#### split loc_of_formation on first comma (or a bracket) and assign 'city', 'admin_area' as appropriate

In [282]:
mask = (
    wd['loc_of_formation'].notna()
    &wd['loc_of_formation'].dropna().str.contains(',')
)
    
wd.update(
    wd.loc[mask,'loc_of_formation']
    .str.split(
        '[(,]',
        n=1,
        expand=True
    ).rename(
        columns={0:'loc_of_formation',
                 1:'admin_area'}))

#### homogenise format

In [283]:
wd['country'] = wd['country'].str.strip()
wd['admin_area'] = wd['admin_area'].str.strip()
wd['genre'] = wd['genre'].str.strip().str.casefold()
wd['loc_of_formation'] = wd['loc_of_formation'].str.strip()

In [284]:
wd = wd.replace(aliases_['country']) # catch country names in 'loc_of_formation' or 'admin_names' as well
wd['loc_of_formation'] = wd['loc_of_formation'].replace(aliases_['city'])

#### Remove bad names

In [285]:
wd.loc[wd['country'].isin(uk_country_names), 'country'] = 'United Kingdom'


In [286]:
# New York, USA becomes New York City, USA
wd.loc[(wd['loc_of_formation']=='New York')&(wd['country']=='USA'),'loc_of_formation'] = 'New York City'

#### if 'loc_of_formation' matches neither 'city_aliases' nor 'country_names' _and_ 'admin_area' is NA, let 'admin_area' = 'loc_of_formation'

In [287]:
mask = (wd['loc_of_formation'].notna()
        &wd['admin_area'].isna()
        &~wd['loc_of_formation'].isin(city_aliases)
        &~wd['loc_of_formation'].isin(country_names))

wd.loc[mask,'admin_area'] = wd.loc[mask,'loc_of_formation']

In [288]:
# Japanese prefectures in 'loc_of_formation' changed to main city instead
mask = wd['loc_of_formation'].notna()&wd['loc_of_formation'].dropna().str.endswith('Prefecture')
wd.loc[mask, 'loc_of_formation'] = wd.loc[mask, 'loc_of_formation'].apply(lambda area : area[:-11])

In [297]:
wd.loc[wd['loc_of_formation'].notna()&wd['country'].notna()&~wd['loc_of_formation'].isin(city_aliases)].head(100)

,artist,loc_of_formation,admin_area,genre,country
2,Kate Bush,English people,English people,NaN,United Kingdom
20,Dean,South Korea,NaN,NaN,South Korea
25,Cho Kyuhyun,South Korea,NaN,NaN,South Korea
27,Vytautas Juozapaitis,Lithuania,NaN,NaN,Lithuania
28,Müslüm Gürses,Şanlıurfa Province,Şanlıurfa Province,NaN,Turkey
56,Kino,United Kingdom,NaN,NaN,Soviet Union
173,Bleizi Ruz,Brittany,Brittany,NaN,France
299,Hillsboro Symphony Orchestra,USA,NaN,NaN,USA
355,Los Sabandeños,Canary Islands,Canary Islands,NaN,Spain
364,Desperta Ferro,Catalan Countries,Catalan Countries,NaN,Spain


#### transpose "admin_area" if it matches a country or city name, and delete if it matches a country name

In [102]:
mask = ((wd['loc_of_formation'].isna()
        |~wd['loc_of_formation'].isin(city_aliases))
        &wd['admin_area'].notna()
        &wd['admin_area'].isin(city_aliases)
        &~wd['admin_area'].dropna().isin(country_names+uk_country_names))
wd.loc[mask,'loc_of_formation'] = wd.loc[mask, 'admin_area']

mask = ((wd['country'].isna())
        &wd['admin_area'].notna()
        &wd['admin_area'].dropna().isin(country_names+uk_country_names))
wd.loc[mask,'country'] = wd.loc[mask, 'admin_area']
wd.loc[mask,'admin_area'] = np.NaN

#### Get admin_area based on existing 'loc_of_formation'+'country' info

In [103]:
# loc_area_country = wd.loc[
#     wd['loc_of_formation']
#     .notna()
#     &wd['admin_area']
#     .notna()
#     &wd['country']
#     .notna()
#     &wd['loc_of_formation'].isin(wd.loc[wd['admin_area'].isna(),'loc_of_formation'].values)
#     &wd['country'].isin(wd.loc[wd['admin_area'].isna(),'country'].values),
#     ['loc_of_formation','admin_area','country']
# ].drop_duplicates(['loc_of_formation','country'], keep=False)

In [104]:
# loccountry_to_admin = dict(zip(loc_area_country['loc_of_formation']+loc_area_country['country'],loc_area_country['admin_area']))

In [105]:
# sub_df = wd.loc[
#     wd['admin_area']
#     .isna()
#     &wd['loc_of_formation']
#     .isin(loc_area_country['loc_of_formation'].values)
#     &wd['country']
#     .isin(loc_area_country['country'].values),
#     ['loc_of_formation','admin_area','country']
# ]

# concat = sub_df['loc_of_formation']+sub_df['country']


# # (wd['loc_of_formation']+wd['admin_area']).replace(locarea_to_country)

In [106]:
# concat.replace(loccountry_to_admin)

In [107]:
# wd.loc[
#     wd['admin_area']
#     .isna()
#     &wd['loc_of_formation']
#     .isin(loc_area_country['loc_of_formation'].values)
#     &wd['country']
#     .isin(loc_area_country['country'].values),
#     'admin_area'
# ] = concat.replace(loccountry_to_admin)

#### Get country based on existing 'loc_of_formation'+'admin_area' info

In [108]:
loc_area_country = wd.loc[
    wd['loc_of_formation']
    .notna()
    &wd['admin_area']
    .notna()
    &wd['country']
    .notna()
    &wd['loc_of_formation'].isin(wd.loc[wd['country'].isna(),'loc_of_formation'].values)
    &wd['admin_area'].isin(wd.loc[wd['country'].isna(),'admin_area'].values),
    ['loc_of_formation','admin_area','country']
].drop_duplicates(['loc_of_formation','admin_area'], keep=False)

In [109]:
locarea_to_country = dict(zip(loc_area_country['loc_of_formation']+loc_area_country['admin_area'],loc_area_country['country']))

In [110]:
sub_df = wd.loc[
    wd['country']
    .isna()
    &wd['loc_of_formation']
    .isin(loc_area_country['loc_of_formation'].values)
    &wd['admin_area']
    .isin(loc_area_country['admin_area'].values),
    ['loc_of_formation','admin_area','country']
]

concat = sub_df['loc_of_formation']+sub_df['admin_area']


# (wd['loc_of_formation']+wd['admin_area']).replace(locarea_to_country)

In [111]:
wd.loc[
    wd['country']
    .isna()
    &wd['loc_of_formation']
    .isin(loc_area_country['loc_of_formation'].values)
    &wd['admin_area']
    .isin(loc_area_country['admin_area'].values),
    'country'
] = concat.replace(locarea_to_country)

#### transpose loc_of_formation and country to other instances of same artist (diff. genres perhaps)

In [112]:
mask = (wd['loc_of_formation'].notna()
       &wd['artist'].str.casefold().isin(wd.loc[wd['loc_of_formation'].isna(),'artist'].str.casefold().values))

arc = wd.loc[mask,['artist','loc_of_formation']].sort_values(['loc_of_formation']).drop_duplicates(['artist'])

artist_to_city = dict(
    zip(
        arc['artist'].str.casefold().values,
        arc['loc_of_formation'].values
    )
)

# mask = (wd['country'].notna()
#        &wd['artist'].str.casefold().isin(wd.loc[wd['country'].isna(),'artist'].str.casefold().values))

# arc = wd.loc[mask,['artist','country']].sort_values(['country']).drop_duplicates(['artist'])

# artist_to_country = dict(
#     zip(
#         arc['artist'].str.casefold().values,
#         arc['country'].values
#     )
# )

In [113]:
mask = (wd['loc_of_formation'].isna()
        &wd['artist'].str.casefold().isin(artist_to_city.keys()))

wd.loc[mask,'loc_of_formation'] = wd.loc[mask,'artist'].str.casefold().replace(artist_to_city)

# mask = (wd['country'].isna()
#         &wd['artist'].str.casefold().isin(artist_to_country.keys()))

# wd.loc[mask,'country'] = wd.loc[mask,'artist'].str.casefold().replace(artist_to_country)

#### get country based on existing 'loc_of_formation' OR 'admin_area' information

In [114]:
# area_to_country = dict(
#     wd.loc[
#         wd['admin_area']
#         .notna()
#         &wd['country']
#         .notna()
#         &wd['admin_area']
#         .isin(wd.loc[
#             wd['country'].isna(),
#             'admin_area'
#         ].values),
#         ['admin_area','country']
#     ].drop_duplicates(['admin_area'], keep=False).values
# )

# mask = wd['country'].isna()&wd['admin_area'].notna()&wd['admin_area'].isin(area_to_country.keys())
# wd.loc[mask, 'country'] = wd.loc[mask, 'admin_area'].replace(area_to_country)

In [115]:
# wd_city_to_country = dict(
#     wd.loc[
#         wd['loc_of_formation']
#         .notna()
#         &wd['country']
#         .notna()
#         &wd['loc_of_formation']
#         .isin(wd.loc[
#             wd['country'].isna(),
#             'loc_of_formation'
#         ].values),
#         ['loc_of_formation','country']
#     ].drop_duplicates(['loc_of_formation','country']).values
# )

# mask = wd['country'].isna()&wd['loc_of_formation'].notna()&wd['loc_of_formation'].isin(wd_city_to_country.keys())
# wd.loc[mask, 'country'] = wd.loc[mask, 'loc_of_formation'].replace(wd_city_to_country)

#### country from external cities or admin areas

In [116]:
# Japan from prefecture
mask = (wd['loc_of_formation'].notna()
        &(wd['loc_of_formation'].dropna().str.endswith('Prefecture')
        |wd['admin_area'].dropna().str.endswith('Prefecture')))

wd.loc[mask, 'country'] = 'Japan'

In [117]:
wd.loc[wd['country'].isna()&((wd['loc_of_formation'].isin(uk_country_names))|(wd['admin_area'].isin(uk_country_names))),'country'] = 'United Kingdom'

In [118]:
wd.loc[wd['country'].isna()&((wd['loc_of_formation'].isin(us_state_names))|(wd['admin_area'].isin(us_state_names))), 'country'] = 'USA'

In [119]:
wd.loc[wd['country'].isna()&((wd['loc_of_formation'].isin(uk_county_names))|(wd['admin_area'].isin(uk_county_names))), 'country'] = 'United Kingdom'

In [120]:
wd.loc[wd['country'].isna()&((wd['loc_of_formation'].isin(us_county_names))|(wd['admin_area'].isin(us_county_names))), 'country'] = 'USA'

In [127]:
mask = wd['country'].isna()&wd['loc_of_formation'].isin(city_to_country.keys())
wd.loc[mask, 'country'] = wd.loc[mask, 'loc_of_formation'].replace(city_to_country)

mask = wd['country'].isna()&wd['admin_area'].isin(city_to_country.keys())
wd.loc[mask, 'country'] = wd.loc[mask, 'admin_area'].replace(city_to_country)

In [128]:
# print('city_to_country: ',len(city_to_country),'\nwd_city_to_country: ',len(wd_city_to_country),'\narea_to_country: ',len(area_to_country))

In [129]:
wd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102369 entries, 0 to 102368
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   artist            102369 non-null  object
 1   loc_of_formation  50532 non-null   object
 2   admin_area        2663 non-null    object
 3   genre             59827 non-null   object
 4   country           92916 non-null   object
dtypes: object(5)
memory usage: 9.7+ MB


#### tidy up where possible

In [144]:
wd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102369 entries, 0 to 102368
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   artist            102369 non-null  object 
 1   loc_of_formation  50532 non-null   object 
 2   admin_area        2663 non-null    object 
 3   genre             59827 non-null   object 
 4   country           91749 non-null   object 
 5   city              0 non-null       float64
dtypes: float64(1), object(5)
memory usage: 10.5+ MB


In [146]:
wd.loc[~wd['country'].isin(country_names), 'country'] = np.nan
wd.loc[~wd['loc_of_formation'].isin(city_aliases), 'loc_of_formation'] = np.nan

In [147]:
city_states = gn.loc[gn['city']==gn['country'], 'city'].unique()
wd.loc[wd['country'].isna()&wd['loc_of_formation'].isin(city_states), 'country'] = wd.loc[wd['loc_of_formation'].isin(city_states), 'loc_of_formation']
wd.loc[wd['loc_of_formation'].isna()&wd['country'].isin(city_states), 'loc_of_formation'] = wd.loc[wd['country'].isin(city_states), 'country']

#### finishing

In [148]:
wd = wd[(~(wd['loc_of_formation']==wd['country']))
        |(wd['loc_of_formation'].isin(city_states))]

In [149]:
wd['admin_area'] = wd['admin_area'].str.strip()

In [150]:
wd = wd.dropna(subset=['loc_of_formation','country'])

In [151]:
wd = wd.drop_duplicates()

In [152]:
wd = wd.rename(columns={'loc_of_formation': 'city'})

In [153]:
wd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42232 entries, 0 to 102289
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   artist      42232 non-null  object
 1   city        42232 non-null  object
 2   admin_area  1090 non-null   object
 3   genre       23053 non-null  object
 4   country     42232 non-null  object
dtypes: object(5)
memory usage: 1.9+ MB


In [154]:
wd.sort_values('city').drop_duplicates('artist').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18968 entries, 517 to 56881
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   artist      18968 non-null  object
 1   city        18968 non-null  object
 2   admin_area  606 non-null    object
 3   genre       8769 non-null   object
 4   country     18968 non-null  object
dtypes: object(5)
memory usage: 889.1+ KB


In [155]:
wd.nunique()

artist        18968
city           2953
admin_area      253
genre           792
country         141
dtype: int64

In [156]:
wd['country'].value_counts(normalize=True)

USA               0.410210
United Kingdom    0.124100
Japan             0.046884
Canada            0.036299
Australia         0.034784
                    ...   
Gambia            0.000024
Tajikistan        0.000024
Iraq              0.000024
Maldives          0.000024
El Salvador       0.000024
Name: country, Length: 141, dtype: float64

### trebi/Spotify

#### (songs)

In [157]:
ts = pd.read_csv('./trebi/csv/songs.csv',
                 sep=';',
                 usecols=[0,2,4],
                 header=0,
                 names=['songID','artist','genre'])

ts['genre'] = ts['genre'].str.strip().str.casefold()

ts['artist'] = ts['artist'].str.split(', ')
ts = ts.explode('artist',ignore_index=True)
ts['artist'] = ts['artist'].str.strip()

ts = ts.drop_duplicates(['artist','genre'])

In [158]:
ts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292566 entries, 0 to 357995
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   songID  292566 non-null  object
 1   artist  292566 non-null  object
 2   genre   292566 non-null  object
dtypes: object(3)
memory usage: 8.9+ MB


In [159]:
ts.nunique()

songID    166627
artist    177656
genre       1496
dtype: int64

In [160]:
song_to_artist = dict(ts[['songID','artist']].drop_duplicates('songID').values)
len(song_to_artist)

166627

#### trebi tag

In [161]:
# tt = pd.read_csv('./trebi/csv/tags.csv', sep=';', usecols=[0,1], names=['songID','genre'])
# tt = tt.drop_duplicates()
# tt = tt.loc[tt['songID'].isin(ts['songID'].values)]
# tt['genre'] = tt['genre'].str.strip().str.casefold()
# tt['artist'] = tt['songID'].replace(song_to_artist)

In [162]:
# tt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47105 entries, 1 to 989696
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   songID  47105 non-null  object
 1   genre   47105 non-null  object
 2   artist  47105 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [163]:
# tt.nunique()

songID    47105
genre      1476
artist    43641
dtype: int64

#### trebi merge

In [164]:
tstt = ts
# pd.merge(ts, tt, how='outer').drop_duplicates(['artist','genre'])

In [165]:
tstt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292566 entries, 0 to 292565
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   songID  292566 non-null  object
 1   artist  292566 non-null  object
 2   genre   292566 non-null  object
dtypes: object(3)
memory usage: 8.9+ MB


In [166]:
tstt.nunique()

songID    166627
artist    177656
genre       1496
dtype: int64

## merge trebi and wikidata

In [167]:
lkey = wd['artist'].str.casefold()
rkey = tstt['artist'].str.casefold()

df = pd.merge(wd,tstt,left_on=lkey,right_on=rkey,how='outer')

In [168]:
df.loc[df['artist_x'].isna(), 'artist_x'] = df.loc[df['artist_x'].isna(), 'artist_y']

In [169]:
df = df.melt(
    ['artist_x','city','admin_area','country','songID'],
    ['genre_x','genre_y'],
    value_name='genre'
).drop(
    'variable',axis=1
).rename(
    columns={'artist_x': 'artist'}
).drop_duplicates(
    ['artist','city','country','genre']
).dropna(
    subset=['city','country','genre'],
    how='all'
)

In [170]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 345329 entries, 0 to 714293
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   artist      345329 non-null  object
 1   city        80383 non-null   object
 2   admin_area  1531 non-null    object
 3   country     80383 non-null   object
 4   songID      322772 non-null  object
 5   genre       323659 non-null  object
dtypes: object(6)
memory usage: 18.4+ MB


### reinterpret some genres

In [171]:
hyphens = df['genre'].str.contains("-", na=False)
nohy = df.loc[~hyphens,'genre'].values
mask = df['genre'].str.replace("-"," ").isin(nohy)
df.loc[hyphens&mask,'genre'] = df.loc[hyphens&mask,'genre'].str.replace("-"," ")

In [172]:
df['genre'] = df['genre'].str.replace("hip hop", "hip-hop")
df['genre'] = df['genre'].str.replace("rock steady", "rocksteady")


In [173]:
demonyms = ['anatolian', 'argentine', 'basque',
            'boston', 'brazilian', 'bulgarian', 'catalan',
            'chilean', 'colombian', 'czech', 'dutch',
            'french', 'hungarian', 'indian',
            'kiwi', 'kraut', 'norwegian', 'peruvian', 'portuguese',
            'romanian', 'russian', 'scottish', 'slovenian', 'spanish',
            'suomi', 'swiss', 'swedish', 'turkish', 'ukrainian', 'venezuelan',
            'welsh', 'yugoslav','slovak','estonian','icelandic',
            'indonesian', 'albuquerque', 'canadian', 'la',
            'vancouver', 'australian', 'perth', 'german', 'brooklyn',
            'vienna', 'portland', 'vegas', 'mexican', 'sheffield',
            'ok', 'rva', 'athens', 'belgian',
            'slc', 'finnish', 'chicago', 'polish', 'seattle',
            'leeds', 'nz', 'thai', 'louisville', 'danish',
            'denver', 'michigan', 'stl', 'kc', 'triangle',
            'greek', 'irish', 'bay area', 'thai', 'pakistani',
            'polynesian', 'latvian', 'arab', 'korean', 'brazilian',
            'taiwanese', 'faroese', 'austrian', 'persian', 'chinese',
            'italian', 'american', 'puerto rican', 'detroit', 'louisiana',
            'new orleans', 'uk', 'british'
           ]

In [174]:
sorted(demonyms)

['albuquerque',
 'american',
 'anatolian',
 'arab',
 'argentine',
 'athens',
 'australian',
 'austrian',
 'basque',
 'bay area',
 'belgian',
 'boston',
 'brazilian',
 'brazilian',
 'british',
 'brooklyn',
 'bulgarian',
 'canadian',
 'catalan',
 'chicago',
 'chilean',
 'chinese',
 'colombian',
 'czech',
 'danish',
 'denver',
 'detroit',
 'dutch',
 'estonian',
 'faroese',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'icelandic',
 'indian',
 'indonesian',
 'irish',
 'italian',
 'kc',
 'kiwi',
 'korean',
 'kraut',
 'la',
 'latvian',
 'leeds',
 'louisiana',
 'louisville',
 'mexican',
 'michigan',
 'new orleans',
 'norwegian',
 'nz',
 'ok',
 'pakistani',
 'persian',
 'perth',
 'peruvian',
 'polish',
 'polynesian',
 'portland',
 'portuguese',
 'puerto rican',
 'romanian',
 'russian',
 'rva',
 'scottish',
 'seattle',
 'sheffield',
 'slc',
 'slovak',
 'slovenian',
 'spanish',
 'stl',
 'suomi',
 'swedish',
 'swiss',
 'taiwanese',
 'thai',
 'thai',
 'triangle',
 'turkish',
 'uk',
 '

In [176]:
hiphop_mask = df['genre'].str.contains('hip-hop',na=False)
for nym in demonyms:
    mask = df['genre'].str.contains(nym,na=False)&~hiphop_mask
    match = df.loc[mask,'genre'].str.extract('(.*)('+nym+'[ -])(.*)')
    df.loc[mask,'genre'] = (match[0]+match[2]).str.strip()

In [177]:
mask = df['genre'].str.contains("music$",na=False)

df.loc[mask,'genre'] = df.loc[mask,'genre'].str.extract("(.*)([ -]music$)")[0]

In [178]:
df.loc[df['genre'].str.contains('(?<!deep) indie$',na=False),'genre'] = 'indie'

df.loc[df['genre'].notna()&df['genre'].str.contains(r'electronic dance'), 'genre'] = 'edm'

df.loc[df['genre'].str.contains('[ -]singer[- ]songwriter',na=False),'genre'] = 'singer-songwriter'

In [179]:
df = df.drop_duplicates(['artist','genre'])

In [180]:
df.loc

### fill in cities

In [182]:
mask = df['city'].isna()&df['artist'].str.casefold().isin(artist_to_city.keys())
df.loc[mask,'city'] = df.loc[mask,'artist'].str.casefold().replace(artist_to_city)

# mask = df['country'].isna()&df['artist'].str.casefold().isin(artist_to_country.keys())
# df.loc[mask,'country'] = df.loc[mask,'artist'].str.casefold().replace(artist_to_country)

In [183]:
df = df.loc[(df['city'].isin(city_aliases))]

In [184]:
df = df.drop_duplicates(['artist','genre'])

In [185]:
df = df.dropna(subset=['genre','city'])

In [186]:
# mask = df['country'].isna()&df['city'].isin(wd_city_to_country.keys())
# df.loc[mask, 'country'] = df.loc[mask, 'city'].replace(wd_city_to_country)

In [187]:
mask = df['country'].isna()&df['city'].isin(city_to_country.keys())
df.loc[mask, 'country'] = df.loc[mask, 'city'].replace(city_to_country)

In [188]:
df.loc[df['country'].isna()].value_counts('city')

city
Canada                          27
Special Region of Yogyakarta     4
Plzeň                            3
Brașov                           2
Thessaloniki                     1
Portrane                         1
Darjeeling                       1
Chișinău                         1
Aarhus                           1
dtype: int64

In [189]:
df = df.dropna(subset=['country'])

In [190]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43766 entries, 37 to 627664
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   artist      43766 non-null  object
 1   city        43766 non-null  object
 2   admin_area  526 non-null    object
 3   country     43766 non-null  object
 4   songID      34452 non-null  object
 5   genre       43766 non-null  object
dtypes: object(6)
memory usage: 2.3+ MB


In [191]:
df.nunique()

artist        16614
city           2717
admin_area      150
country         125
songID        10784
genre          1352
dtype: int64

In [192]:
failed = []
for city, country in df[['city','country']].drop_duplicates().values:
    try:
        df.loc[
            (df['city']==city)
            &(df['country']==country),
            ['lat','lng','population']
        ] = gn.loc[
            (gn['alternateNames'].str.casefold().str.contains(city.casefold(),na=False))
            &(gn['country'].str.casefold()==country.casefold()),
            ['lat','lng', 'population']
        ].values[0]
    except:
        failed.append(tuple([city, country]))
len(failed)

/Users/Nathan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [194]:
df = df.dropna(subset=['lat','lng','population'])

In [195]:
df = df.loc[df['population']>0.0]

In [201]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41160 entries, 37 to 627035
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   artist      41160 non-null  object 
 1   city        41160 non-null  object 
 2   admin_area  447 non-null    object 
 3   country     41160 non-null  object 
 4   songID      32407 non-null  object 
 5   genre       41160 non-null  object 
 6   lat         41160 non-null  float64
 7   lng         41160 non-null  float64
 8   population  41160 non-null  float64
dtypes: float64(3), object(6)
memory usage: 3.1+ MB


In [202]:
df.value_counts('genre')

genre
rock                    1904
alternative rock        1769
punk rock               1567
indie rock              1316
pop                     1127
                        ... 
japanese traditional       1
jazz bass                  1
jazz brass                 1
jazz composition           1
abstract                   1
Length: 1329, dtype: int64

In [203]:
df.nunique()

artist        15688
city           2393
admin_area      136
country         125
songID        10165
genre          1329
lat            2441
lng            2445
population     2429
dtype: int64

## Map

In [204]:
import folium

from folium.plugins import marker_cluster

In [261]:
# genre = 'k-pop'
# if genre in tlgs:
#     value_counts = df.loc[df['genre'].str.contains(genre,na=False)].drop_duplicates(['artist']).value_counts(['city','country','lat','lng','population'])
# else:
#     value_counts = df.loc[df['genre']==genre].drop_duplicates(['artist']).value_counts(['city','country','lat','lng','population'])

# value_counts = value_counts.loc[value_counts > 5]
# total_genre_bands = value_counts.sum()

# bands_per_million = (value_counts / (value_counts.index.get_level_values('population') / 1000000))
# people_per_band = (value_counts.index.get_level_values('population') / value_counts)
# one_in_x = (city_band_count[value_counts.index] / value_counts)

# city, country, la, lo, po = one_in_x.idxmin()
# one_in_x = one_in_x.min()

# count = value_counts[city,country,la,lo,po]
# bands_per_million = bands_per_million[city,country,la,lo,po]
# people_per_band = people_per_band[city,country,la,lo,po]
# all_bands_here = city_band_count[city,country]

In [211]:
homes_of = []
failed = []

letters = '[^\s'+punctuation.replace('&','')+']+'
word = letters+'-?'+letters
tlg_mask = df['genre'].str.fullmatch(letters+'-?'+letters,na=False)
tlgs = df.loc[tlg_mask,'genre'].value_counts()
tlgs = tlgs.loc[tlgs > 12].index

city_band_count = df.drop_duplicates('artist').value_counts(['city','country'])

for genre in df['genre'].value_counts().loc[df['genre'].value_counts() > 12].index.tolist():
    if genre in tlgs:
        value_counts = df.loc[df['genre'].str.contains(genre,na=False)].drop_duplicates(['artist']).value_counts(['city','country','lat','lng','population'])
    else:
        value_counts = df.loc[df['genre']==genre].drop_duplicates(['artist']).value_counts(['city','country','lat','lng','population'])

    value_counts = value_counts.loc[value_counts > 5]
    total_genre_bands = value_counts.sum()
    
    if len(value_counts) == 0:
        failed.append(genre)
        continue
        
    bands_per_million = (value_counts / (value_counts.index.get_level_values('population') / 1000000))
    people_per_band = (value_counts.index.get_level_values('population') / value_counts)
    one_in_x = (city_band_count[value_counts.index] / value_counts)
    
    city, country, la, lo, po = one_in_x.idxmin()
    one_in_x = one_in_x.min()
    
    count = value_counts[city,country,la,lo,po]
    bands_per_million = bands_per_million[city,country,la,lo,po]
    people_per_band = people_per_band[city,country,la,lo,po]
    all_bands_here = city_band_count[city,country]
    
    
    homes_of.append(tuple(
        [
            genre,
            count,
            city,
            country,
            bands_per_million,
            people_per_band,
            one_in_x,
            all_bands_here,
            total_genre_bands,
            la.round(5),
            lo.round(5)
        ]
    ))

In [212]:
len(homes_of)

244

In [213]:
len(failed)

202

In [243]:
m2 = folium.Map(location=[51, 0.3],
              zoom_start=5)

locations = []
popups = []

for ge, num, ci, co, bpm, ppb, oix, abh, tgb, la, lo in homes_of:
    
    sd = -(len(str(int(ppb)))-2)

    popup = folium.Popup(
        f"<p style='white-space:nowrap'>{ci} ({co}) is the modern home of: <strong>{ge}</strong><br/>\
        1 in {int(round(oix,0))} bands from {ci} is a {ge} band (one for every ~{int(round(ppb, sd)):n} people)<br/><br/>\
        <em>In this dataset, there are…\
        <ul><li>{num} {ge} bands from {ci} ({int(round(num / tgb * 100, 0))}% of all {ge} bands)\
        <li>{abh} bands from {ci} in total\
        <li>{tgb} {ge} bands in total</em><ul></p>",
        sticky=True)

    locations.append([la,lo])
    popups.append(popup)
        
cluster = marker_cluster.MarkerCluster(locations,popups)
        
#         folium.Marker(
#             location=[la,lo],
#             tooltip=f'{ci} ({co}) is the home of: {ge}<br/>with {num} {ge} band(s)',
#             popup=popup,
#             icon=folium.Icon(icon_color='white')
#         ).add_to(m2)

m2.add_child(cluster)